<a href="https://colab.research.google.com/github/linqus/nlp-huggingface/blob/main/notebooks/unit7/nlp_7_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets transformers[sentencepiece] sacremoses accelerate

In [4]:
from datasets import load_dataset

raw_datasets = load_dataset('kde4', lang1='en', lang2='fr')

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [5]:
split_datasets = raw_datasets['train'].train_test_split(test_size=0.1, shuffle=True, seed=20)
split_datasets['validation'] = split_datasets.pop('test')
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [6]:
split_datasets['train'][1]

{'id': '152754',
 'translation': {'en': 'Default to expanded threads',
  'fr': 'Par défaut, développer les fils de discussion'}}

In [7]:
from transformers import pipeline

model_checkpoint = 'Helsinki-NLP/opus-mt-en-fr'

translator = pipeline('translation', model_checkpoint)
outputs = translator(split_datasets['train'][1]['translation']['en'])
outputs

[{'translation_text': 'Par défaut pour les threads élargis'}]

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors='pt')

en_inputs = tokenizer(split_datasets['train'][1]['translation']['en'])
en_inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [9]:
wrong_inputs = tokenizer(split_datasets['train'][1]['translation']['fr'])


In [10]:
full_inputs = tokenizer(split_datasets['train'][1]['translation']['en'], text_target=split_datasets['train'][1]['translation']['fr'])
full_inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [11]:
print('wrong fr: ', tokenizer.convert_ids_to_tokens(wrong_inputs['input_ids']))
print('good fr: ', tokenizer.convert_ids_to_tokens(full_inputs['labels']))

wrong fr:  ['▁Par', '▁dé', 'f', 'aut', ',', '▁dé', 've', 'lop', 'per', '▁les', '▁fil', 's', '▁de', '▁discussion', '</s>']
good fr:  ['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']


In [12]:
max_length=128

def process_sentences(examples):

  en_sentences = [ex['en'] for ex in examples['translation']]
  fr_sentences = [ex['fr'] for ex in examples['translation']]
  result = tokenizer(en_sentences, text_target=fr_sentences, max_length=128, truncation=True)

  return result

In [13]:
samples = [split_datasets['train'][i] for i in range(3)]
samples = split_datasets['train'][:3]
samples

{'id': ['92924', '152754', '164335'],
 'translation': [{'en': "Calibration is about to check the value range your device delivers. Please move axis %1 %2 on your device to the maximum position. Press any button on the device or click on the'Next 'button to continue with the next step.",
   'fr': "Le calibrage va vérifier la plage de valeurs que votre matériel produit. Veuillez déplacer l'axe %1 %2 de votre périphérique à la position maximale. Appuyez sur n'importe quel bouton du périphérique ou sur le bouton « & #160; Suivant & #160; » pour la prochaine étape."},
  {'en': 'Default to expanded threads',
   'fr': 'Par défaut, développer les fils de discussion'},
  {'en': '*. ui *. UI_BAR_User Interface Files',
   'fr': '*. ui *. UI_BAR_Fichiers interface utilisateur'}]}

In [14]:
process_sentences(samples)

{'input_ids': [[34378, 226, 5783, 32, 200, 12, 3647, 4, 1223, 1628, 117, 4923, 23608, 3, 1789, 2942, 20059, 301, 548, 301, 331, 30, 117, 4923, 12, 4, 1528, 668, 3, 5734, 212, 9319, 30, 4, 4923, 57, 5487, 30, 4, 6, 32712, 25, 7243, 1160, 12, 621, 42, 4, 1156, 3009, 3, 0], [47591, 12, 9842, 19634, 9, 0], [1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 28149, 139, 33712, 25218, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[60, 7418, 5244, 8234, 740, 4993, 8, 6471, 5, 2218, 29, 193, 2220, 742, 3, 4366, 14237, 14, 6, 16600, 301, 548, 301, 331, 5, 193, 24275, 17, 8, 668, 6142, 3, 33640, 36, 81, 6, 5411, 2709, 9376, 22, 24275, 59, 36, 19, 9376, 153, 402, 29033, 13774, 402, 29033, 416, 27, 8, 4034, 4888, 3, 0], [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0], [1211, 3, 49, 9409, 121

In [15]:
tokenized_datasets = split_datasets.map(process_sentences, batched=True, remove_columns=['id', 'translation'])
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21018
    })
})

In [16]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)



In [17]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer

data_collator = DataCollatorForSeq2Seq(tokenizer, model)

inputs = data_collator([tokenized_datasets['train'][i] for i in range(3)])
inputs


{'input_ids': tensor([[34378,   226,  5783,    32,   200,    12,  3647,     4,  1223,  1628,
           117,  4923, 23608,     3,  1789,  2942, 20059,   301,   548,   301,
           331,    30,   117,  4923,    12,     4,  1528,   668,     3,  5734,
           212,  9319,    30,     4,  4923,    57,  5487,    30,     4,     6,
         32712,    25,  7243,  1160,    12,   621,    42,     4,  1156,  3009,
             3,     0],
        [47591,    12,  9842, 19634,     9,     0, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
         59513, 59513],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
         28149,   139, 33712, 25218,     0, 59513, 59513, 59513, 5951

In [18]:
inputs['input_ids'].shape

torch.Size([3, 52])

In [19]:
inputs['decoder_input_ids'].shape

torch.Size([3, 58])

In [20]:
!pip install sacrebleu evaluate

In [21]:
import evaluate

metric =evaluate.load('sacrebleu')


In [22]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]

In [23]:
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [24]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [25]:
predictions = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 0.004086771438464067,
 'sys_len': 2,
 'ref_len': 13}

In [26]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [27]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [28]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [29]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 1.696446418762207,
 'eval_bleu': 39.166011386870395,
 'eval_runtime': 1565.1179,
 'eval_samples_per_second': 13.429,
 'eval_steps_per_second': 0.21}

In [ ]:
trainer.train()

Step,Training Loss
500,1.386400
1000,1.230700
1500,1.195500
2000,1.137700
2500,1.111000
3000,1.079100
3500,1.044200
4000,1.032000
4500,1.039100
5000,1.010000


In [ ]:
trainer.evaluate(max_length=max_length)

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")